In [1]:
import scCube
from scCube import scCube
import pandas as pd
import numpy as np
import warnings
import time
warnings.filterwarnings("ignore")

/home/qjy/anaconda3/envs/sccube/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = scCube()

In [3]:
epochs = 50000
bregma_list = ['n0.29', 'n0.24', 'n0.19', 'n0.14', 'n0.09', 'n0.04', '0.01', '0.06', '0.11', '0.16', '0.21', '0.26'] 
running_time = []

In [ ]:
for i in bregma_list:
    print('Training model using slice: ' + str(i) + ' data...')
    sc_data = pd.read_csv('data/MERFISH/processed/Animal1_Bregma_' + str(i) + '_data.csv', index_col=0)
    sc_meta = pd.read_csv('data/MERFISH/processed/Animal1_Bregma_' + str(i) + '_meta.csv', index_col=0)
    
    sc_adata = model.pre_process(
        sc_data=sc_data, 
        sc_meta=sc_meta,
        is_normalized=False)
    
    model_name = 'bregma_' + str(i) + '_epoch50000'
    
    # train
    t1 = time.time()
    generate_sc_meta, generate_sc_data = model.train_vae_and_generate_cell(
        sc_adata=sc_adata,
        celltype_key='Cell_type',
        cell_key='Cell',
        target_num=None,
        batch_size=512,
        epoch_num=epochs,
        lr=0.0001,
        hidden_size=128,
        save_model=True,
        save_path='/home/qjy/workspace/scCube/save_model',
        project_name=model_name,
        used_device='cuda:0')
    t2 = time.time()
    
    running_time.append(t2 - t1)
    
    # generate
    t3 = time.time()
    generate_sc_data, generate_sc_meta = model.generate_pattern_reference(
        sc_adata=sc_adata,
        generate_sc_data=generate_sc_data,
        generate_sc_meta=generate_sc_meta,
        celltype_key='Cell_type',
        spatial_key=['x', 'y'],
        cost_metric='sqeuclidean'
    )
    t4 = time.time()
    running_time.append(t4 - t3)
    
    # saving
    generate_sc_data.to_csv('result/sccube_MERFISH_' + str(i) + '_epoch50000_data.csv')
    generate_sc_meta.to_csv('result/sccube_MERFISH_' + str(i) + '_epoch50000_meta.csv')

Training model using slice: n0.29 data...
the input is count matrix, normalizing it firstly...
generating by the proportion of cell types of the input scRNA-seq data...
begin vae training...


Train Epoch: 34201:  68%|██████▊   | 34201/50000 [33:04<15:19, 17.18it/s, loss=4.8795, min_loss=4.8668]

In [5]:
bregma_list_all = list(np.repeat(bregma_list, 2))
mode = ['train', 'generate']*12

running_time_df = pd.DataFrame({'bregma': bregma_list_all, 'mode': mode, 'time':running_time})
running_time_df.to_csv('result/sccube_MERFISH_self_time_epoch50000.csv')

running_time_df

,bregma,mode,time
0,n0.29,train,2894.465940
1,n0.29,generate,1.183075
2,n0.24,train,2874.672482
3,n0.24,generate,1.103978
4,n0.19,train,3076.192426
5,n0.19,generate,1.290024
6,n0.14,train,3115.954915
7,n0.14,generate,1.470842
8,n0.09,train,2872.800409
9,n0.09,generate,1.118522
